# DecisionTree Assignment - 20기 OOO

물음표 친 부분을 채우고 코드에 대한 주석을 자세하게 달아주세요!

# Data Loading

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [3]:
def get_gini(df, label):
     # 클래스 빈도수 계산
    freq = df[label].value_counts(normalize=True)

    # 지니 계수 계산
    gini = 1 - sum(freq**2)

    return gini

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [9]:
from itertools import combinations

def get_binary_split(df, attribute):

    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트

    result = []

    for r in range(1, len(uniques)):
        for comb in combinations(uniques, r):
            result.append(list(comb))

    return result


In [11]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [12]:
def get_attribute_gini_index(df, attribute, label):

    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)

    for split in binary_split:
        # first
        subset_1 = df[df[attribute].isin(split)]
        gini_1 = get_gini(subset_1, label)

        # second
        subset_2 = df[~df[attribute].isin(split)]
        gini_2 = get_gini(subset_2, label)

        # weighted sum
        weighted_gini = (len(subset_1) / n) * gini_1 + (len(subset_2) / n) * gini_2

        result[tuple(split)] = weighted_gini

    return result

In [13]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{('high',): 0.4428571428571429,
 ('medium',): 0.4583333333333333,
 ('low',): 0.45,
 ('high', 'medium'): 0.45,
 ('high', 'low'): 0.4583333333333333,
 ('medium', 'low'): 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [14]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

(('high',), 0.4428571428571429)

In [15]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

('high',)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [19]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

min_gini = 9999.
important_feature = ''

for feature in features:
    gini_values = get_attribute_gini_index(pd_data, feature, label)
    current_min_gini = min(gini_values.values())

    # 해당 변수의 최소 gini 계수값 출력
    print(f"Mimimum Gini Index of {feature} : {current_min_gini:.4f}")

Mimimum Gini Index of age : 0.3571
Mimimum Gini Index of income : 0.4429
Mimimum Gini Index of student : 0.3673
Mimimum Gini Index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [20]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.45714285714285713,
 ('youth', 'middle_aged'): 0.45714285714285713,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [25]:
from math import log2

def getEntropy(df, feature) :
    freq = df[feature].value_counts(normalize=True)

    entropy = -sum([p * log2(p) for p in freq])

    return entropy

In [27]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [32]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :

    result = {}

    # 엔트로피 계산
    total_entropy = getEntropy(df, feature)

    for c in df.columns:
        # 인자로 받은 feature 제외
        if c == feature:
            continue

        weighted_entropy = 0

        # weighted sum
        for value in df[c].unique():
            subset = df[df[c] == value]
            weight = len(subset) / len(df)
            weighted_entropy += weight * getEntropy(subset, feature)

        # 정보 획득량 계산
        gain = total_entropy - weighted_entropy
        result[c] = gain

    return result

In [33]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}